In [ ]:
# InterpolationCodeUsage.ipynb testing

In [1]:
from utility_programs.read_routines import SAMI

from scipy.spatial import Delaunay
from scipy.interpolate import LinearNDInterpolator, griddata
import numpy as np
import pandas as pd


from datetime import datetime
from tqdm.auto import tqdm
import xarray as xr

import matplotlib.pyplot as plt

import cartopy.crs as ccrs

#import test_interpolate_outputs

!pwd

/home/pxv220016/prasoon/data/SAMI3-GITM-python


In [ ]:
# !chmod -R 770 test


In [2]:
import pandas as pd

allsatdf = pd.read_csv('/petastore/phil/Prasoon_satellite_project/satfiles/20130317/DMSP_all-with_datetime.csv')

firstsat = allsatdf.loc[allsatdf.sat == 16].reset_index(drop=True)
# make sure to reset index & drop bad values, otherwise it's harder
firstsat


,glat,glon,time,sat,dayofyear,dtime
0,2.8,261.2,0.0,16,75,2013-03-16 00:00:00
1,6.3,260.5,60.0,16,75,2013-03-16 00:01:00
2,9.8,259.7,120.0,16,75,2013-03-16 00:02:00
3,13.3,258.9,180.0,16,75,2013-03-16 00:03:00
4,16.8,258.0,240.0,16,75,2013-03-16 00:04:00
...,...,...,...,...,...,...
4315,56.7,96.1,86100.0,16,77,2013-03-18 23:55:00
4316,53.3,94.3,86160.0,16,77,2013-03-18 23:56:00
4317,49.9,92.7,86220.0,16,77,2013-03-18 23:57:00
4318,46.5,91.2,86280.0,16,77,2013-03-18 23:58:00


In [4]:
latlon = [firstsat.glat, firstsat.glon]
latlon = np.array(latlon)

# no alt in files, just assume it's 840 km. Add that to latlon object
alt = np.array([840]) #[840 for i in range(len(latlon[0]))] # thus everywhere the 840km is set as the altitude

lat = latlon[0]
lon = latlon[1]


# add in alt.
# Theres's a better way to do this but I don't remember...
latlonalt = ([lat[:], lon[:], alt])

latlonalt

[array([ 2.8,  6.3,  9.8, ..., 49.9, 46.5, 43. ]),
 array([261.2, 260.5, 259.7, ...,  92.7,  91.2,  89.9]),
 array([840])]

In [5]:
import interpolate_outputs

# out_path = '/home/pxv220016/prasoon/data/SAMI3-GITM-python/test'
out_path = '/petastore/phil/Prasoon_satellite_project/'

sami_data_path = '/petastore/phil/Prasoon_satellite_project/more_runs/20130317/sami-gitm-coupled/'

ds = interpolate_outputs.do_interpolations(sami_data_path=sami_data_path,
                                     dtime_sim_start='20130316',
                                     out_lat_lon_alt=latlonalt,
                                     out_path=out_path,
                                     out_runname='-at-sat_16',
                                     save_delauney=True,
                                     max_alt=1500, #to make running faster. Not necessary.
                                     cols='edens',
                                    show_progress=True,
                                    engine='h5netcdf', #change to None if this makes errors
                                      return_ds_too=True
                                     )

attempting to reuse existing triangulation file


Reading in SAMI data:   0%|          | 0/596 [00:00<?, ?it/s]

  0%|          | 0/597 [00:00<?, ?it/s]

UnboundLocalError: cannot access local variable 'altout' where it is not associated with a value

In [9]:
# then read the created file with:

!cd test
!pwd

import xarray as xr
import os

ds = xr.open_dataset(os.path.join(out_path, 'SAMI_REGRID-at-sat_16.nc'),
                   engine='h5netcdf') #set engine to the same one used to calculate interpolation.

print(ds)

/home/pxv220016/prasoon/data/SAMI3-GITM-python
<xarray.Dataset>
Dimensions:  (time: 597, lat: 4320, lon: 4320, alt: 1)
Coordinates:
  * time     (time) datetime64[ns] 2011-03-16 ... 2011-03-18T01:39:54
  * lat      (lat) float64 2.8 6.3 9.8 13.3 16.8 ... 56.7 53.3 49.9 46.5 43.0
  * lon      (lon) float64 261.2 260.5 259.7 258.9 258.0 ... 94.3 92.7 91.2 89.9
  * alt      (alt) int64 840
Data variables:
    edens    (time, lat, lon, alt) float64 ...


In [ ]:
import importlib
import interpolate_outputs

In [ ]:
ds = interpolate_outputs.do_interpolate([...], return_ds_too = True)

In [ ]:
ds.edens.sel(alt=300, time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [ ]:
ds.edens.integrate('alt').sel(time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [ ]:
del ds

In [ ]:
ds = xr.open_mfdataset('[.....]SAMI_REGRID.nc')

In [ ]:
ds.edens.sel(alt=300, time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [ ]:
ds.edens.integrate('alt').sel(time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [ ]:
interpolate_outputs = importlib.reload(interpolate_outputs)
interpolate_outputs.do_interpolate([...], return_ds_too = False)
ds = xr.open_mfdataset('[.....]SAMI_REGRID.nc')



In [ ]:
ds.edens.sel(alt=300, time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [ ]:
ds.edens.integrate('alt').sel(time = "YYYMMMDDD HH:MM:SS", method='nearest').plot(x='lon', y='lat')

In [2]:
def do_interpolations(
    sami_data_path=None,
    dtime_sim_start=None,
    gitm_data_path=None,
    gitm_output_each_var=True,
    gitm_output_each_time=False,
    out_lat_lon_alt=None,
    out_path=None,
    out_runname='',
    save_delauney=False,
    max_alt=None,
    cols='all',
    show_progress=False,
    engine='h5netcdf',
    return_ds_too=False,
):
    """Interpolate SAMI (GITM functionality not done yet) to either a
        standard geographic grid or to user-defined points.

    Args:
        sami_data_math (string): path to sami data.
        dtime_sim_start (string/datetime.datetime): Start time of simulation.
            Required to read SAMI data. Can be str (YYYYMMDD) or a pre-computed
            datetime object.
        gitm_data_path (string): path to gitm data.
        gitm_output_each_var (bool): If True, output each variable to a
            separate file. Requires looping through the GITM output files
            multiple times. If False, gitm_output_each_time must be True.
        gitm_output_each_time (bool): If True, output each time to a separate
            file. Will run faster for all variables than gitm_output_each_var,
            but will include variables the user does not care for.
        out_lat_lon_alt (numpy.array): Coordinates to interpolate to.
            Must have dimenstions 3xN, where N is number of points.
            Will be converted to cartesian coordinates.
            Lon and Lat in degrees, Alt in km above earth surface.
        out_path (str): Path to save regridded to.
            Default is same as MODEL_data_path.
        out_runname (str): Descriptive name for output file. Appended to
            out_path + SAMI_REGRID.
        save_delauney (bool): Option to save/read delauney weights from file.
            It takes a while to compute them. Weight file is saved to the
            sami_data_path path with a specification of the max_alt.
            Setting to True allows the program to read weights as well as save
            them.
        max_alt (int): specify maximum altitude of data grid to feed in to
            delauney calculations. Useful if you don't want to recalculate
            weights and the interpoaltion is different from one already done.
        cols (str/list-like): Which variables to interpolate. Default is 'all'.
            Can be any str from
            utility_programs.read_routines.SAMI.sami_og_vars.
        show_progress (bool): Show progressbars? Default: False
            Requires tqdm.
        engine (str): Which engine to use when writing netcdf files.
            Default is 'h5netcdf' but can cause some issues on some systems and
            some python environments. Set to None to use default xarray engine.
        return_ds_too (bool): Set to True to also return the interpolated
            dataset.
            !! Does not support multiple variables.
            !! ONLY works for SAMI (currently).


    Returns:
        Nothing. Unless return_ds_too == True
            - The interpolated data is written to a file.
    """
    if sami_data_path is not None and gitm_data_path is not None:
        raise ValueError('Only one of sami_data_path or gitm_data_path can be'
                         ' specified at a time.')

    if out_path is None:
        if sami_data_path is not None:
            out_path = sami_data_path
        else:
            out_path = gitm_data_path

    # outputs...
    if out_lat_lon_alt is None:
        latout = np.arange(-90, 90, 2)
        lonout = np.arange(0, 360, 5)
        if sami_data_path is not None:
            altout = np.arange(200, 2200, 50)
        else:
            altout = np.arange(120, 670, 50)
        out_lats = []
        out_lons = []
        out_alts = []

        for a in latout:
            for o in lonout:
                for l1 in altout:
                    out_lats.append(a)
                    out_lons.append(o)
                    out_alts.append(l1)

        out_lat_lon_alt = latlonalt_to_cart(
            out_lats, out_lons, np.array(out_alts) + 6371)
    else:
<<<<<<< HEAD
        out_lat_lon_alt = latlonalt_to_cart(
            out_lat_lon_alt[0], out_lat_lon_alt[1],
            np.array(out_lat_lon_alt[2]) + 6371)
=======
        latout = np.array(out_lat_lon_alt[0])
        lonout = np.array(out_lat_lon_alt[1])
        altout = np.array(out_lat_lon_alt[2])

    out_lats = []
    out_lons = []
    out_alts = []

    for a in latout:
        for o in lonout:
            for l1 in altout:
                out_lats.append(a)
                out_lons.append(o)
                out_alts.append(l1)

    out_lat_lon_alt = gps_to_ecef_custom(
        out_lons, out_lats, out_alts)
>>>>>>> 548a59d664ddd861776d5dfd38fafee4f0fddf7f

    # deal with sami first
    if sami_data_path is not None:

        if isinstance(dtime_sim_start, str):
            dtime_sim_start = str_to_ut(dtime_sim_start)

        nz, nf, nlt, nt = SAMI.get_grid_elems_from_parammod(sami_data_path)
        # old_shape = [nlt, nf, nz]
        grid = SAMI.get_sami_grid(sami_data_path, nlt, nf, nz)

        # specify max alt to build delauney at
        if max_alt is None:
            if out_lat_lon_alt is not None:
                # alt will be the biggest coord:
                max_alt = np.max(out_lat_lon_alt) + 300
                # add 300 to make sure we have enough points above top
            else:
                max_alt = 2500

        mask = np.where(grid['alt'] < max_alt)
        grid2 = {}
        for k in grid.keys():
            grid2[k] = grid[k][mask].flatten()
        del grid

        in_cart = gps_to_ecef_custom(grid2['glon'],
                                    grid2['glat'],
                                    grid2['alt']).T

        if os.path.exists(os.path.join(sami_data_path,
                                       'delauney_max-%i.pkl' % max_alt)):
            if save_delauney:
                print('attempting to reuse existing triangulation file')
                with open(os.path.join(
                        sami_data_path, 'delauney_max-%i.pkl' % max_alt),
                        'rb') as f:
                    tri = pickle.load(f)
            else:
                print('Found existing triangulation file. Recalculating...',
                      '\n(Specify save_delauney=True to reuse)')
                tri = Delaunay(in_cart)
        else:
            print('Calculating Delauney Triangulation..')
            tri = Delaunay(in_cart)
            if save_delauney:
                print('Saving')
                with open(os.path.join(sami_data_path,
                                       'delauney_max-%i.pkl' % max_alt),
                          'wb') as f:
                    pickle.dump(tri, f)

        # format 'cols' variable
        if cols == 'all':
            cols = SAMI.sami_og_vars.items()
        else:
            if isinstance(cols, str):
                cols = [cols]
            else:
                cols = np.asarray(cols)

        if show_progress:
            pbar = tqdm(total=len(cols) * nt,
                        desc='Reading in SAMI data')

        first = True  # for choosing which mode to write
        for data_var in cols:
            interpd = []
            data, times = SAMI.read_to_nparray(
                sami_data_path, dtime_sim_start,
                cols=data_var,
                skip_time_check=True)

            if show_progress:
                pbar.set_description('interpolating')
            for t in range(len(times)):
                interp = LinearNDInterpolator(
                    tri,
                    data['data'][data_var][:, :, :, t][mask].flatten())
                interpd.append(interp(out_lat_lon_alt.T))
                if show_progress:
                    pbar.update()
            if show_progress:
                pbar.set_description('writing Dataset...')
            ds = xr.Dataset(coords={
                'time': (['time'], times),
                'alt': (['alt'], altout),
                'lat': (['lat'], latout),
                'lon': (['lon'], lonout)},)
            ds[data_var] = (('time', 'lat', 'lon', 'alt'),
                            np.array(interpd).reshape(
                len(times),
                len(latout),
                len(lonout),
                len(altout)))
            if out_runname != '':
                out_runname = '_' + out_runname + '_'
            ds.to_netcdf(os.path.join(
                out_path, 'SAMI_REGRID' + out_runname + '.nc'),
                engine=engine,
                mode='w' if first else 'a',
                encoding={'time': {'dtype': float}})
            print('done')
            first = False
            if return_ds_too:
                return ds
            del ds, interpd, data  # clean up memory

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 107)

In [3]:

do_interpolate([...], return_ds_too = False)
ds = xr.open_mfdataset('[.....]SAMI_REGRID.nc')



NameError: name 'do_interpolate' is not defined